In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [3]:
# Directories for dataset
TRAIN_DIR = "E:\\college\\grad project\\integrated\\training"
TEST_DIR = "E:\\college\\grad project\\integrated\\testing"
MODEL_SAVE_PATH = "saved_model/mri_classifier.keras"
CUSTOM_CNN_SAVE_PATH = "saved_model/custom_cnn_classifier.keras"

In [5]:
# Preprocessing function for training data
def preprocess_training_data(train_dir, img_size=(224, 224)):
    datagen = ImageDataGenerator(
        rescale=1.0/255.0,  # Normalize pixel values
        rotation_range=30,  # Increased rotation for augmentation
        width_shift_range=0.3,  # More aggressive augmentation
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        validation_split=0.2  # Split training data for validation
    )

    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=img_size,S
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )

    val_generator = datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, val_generator

In [7]:
# Preprocessing function for testing data
def preprocess_testing_data(test_dir, img_size=(224, 224)):
    test_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Only normalization for test data
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=img_size,
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )
    return test_generator

In [9]:
# Validate generator outputs
def validate_generator(generator):
    sample_batch = next(iter(generator))
    if len(sample_batch) != 2:
        raise ValueError(f"Generator yielded an element with length {len(sample_batch)}. Expected length is 2: (images, labels).")
    return True

In [11]:
# Load Data
train_gen, val_gen = preprocess_training_data(TRAIN_DIR)
test_gen = preprocess_testing_data(TEST_DIR)

Found 9345 images belonging to 4 classes.
Found 2333 images belonging to 4 classes.
Found 1705 images belonging to 4 classes.


In [13]:
# Validate Data Generators
validate_generator(train_gen)
validate_generator(val_gen)
validate_generator(test_gen)

True

In [15]:
# Calculate Class Weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

Class Weights: {0: 0.9578720787207872, 1: 0.9496951219512195, 2: 1.2023932063818836, 3: 0.9333799440671194}
